<a href="https://colab.research.google.com/github/mglambert/Fundamentos_procesamiento_imagenes/blob/main/2_transformaciones_intensidad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformaciones de intensidad

### Librerias necesarias para este notebook

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
from skimage.morphology import remove_small_holes, erosion, square
import warnings
import cv2
warnings.filterwarnings("ignore")

### Funcion para visualizar 

In [ ]:
def imshow(im, titulo='', cmap=None, rango=(None, None)):
    plt.figure()
    plt.imshow(im, vmin=rango[0], vmax=rango[1], cmap=cmap)
    plt.title(r'{}'.format(titulo), fontdict={'fontsize': 24})
    plt.axis('off')
    plt.show()

### Descargar imagen a utilizar

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1jC3dYUO2KMqF66HTIi07qJkECtAeGKTd' -O im1.jpg

im = plt.imread('im1.jpg')
imshow(im, 'Original');

## Negativo

Las imagenes estan codificadas en un rango de bits, normalmente 8 bits. Esto genera un rango de color entre 0 y 255. Dado un valor $x$ su negativo se define como $255-x$.

In [ ]:
im2 = 255 - im
imshow(im2, 'Negativo')

## Log

La formula general de este tipo de transformacion es:

$$ s = c \cdot log(1+r) $$

donde $c$ es una constante y se asume $r\geq0$.

Este tipo de transformacion se utiliza para expandir el rango de los valores bajos (oscuros) y comprimir los valores altos (claros o intensos).



In [ ]:
im2 = 45.9859 * np.log(1 + np.float64(im))
im2 = np.uint8(im2)
imshow(im2, 'Log')

## AntiLog

Esta transformacion en la inversa de la anterior, definida como:

$$ s = e^{r/c}-1 $$

donde $c$ es una constante y se asume $r\geq0$.

Este tipo de transformacion se utiliza para expandir el rango de los valores altos (claros o intensos) y comprimir los valores bajos (oscuros).


In [ ]:
im2 = np.exp(np.float64(im)/45.9859)-1
im2 = np.uint8(im2)
imshow(im2, 'AntiLog')

## Correccion gamma

La transformacion gamma es una transformacion de potencia definida como:

$$ s = c \cdot r^\gamma$$ 

donde $c$ y $\gamma$ son constantes positivas. 

La correcion de gamma es la responsable de la “sensacion” de contraste de una imagen, y decimos sensación porque no es algo inherente a la imagen, si no a los dispositivos que la registran, a los dispositivos que la reproducen y a la luminosidad del entorno donde la estamos viendo. 

In [ ]:
im2 = 255*(np.float64(im)/255)**0.67
im2 = np.uint8(im2)
imshow(im, 'Correccion $\gamma=0.67$')

im2 = 255*(np.float64(im)/255)**1.5;
im2 = np.uint8(im2);
imshow(im2, 'Correccion $\gamma=1.5$')

### Correccion gamma ejemplo de clases


In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1EG3VjKzd11fJH-DMXvmkaT5d48TVutcq' -O intensity_ramp.tif


im = plt.imread('intensity_ramp.tif');
imshow(im, 'Rampa de intensidad', 'gray');

im2 = 255*(np.float64(im)/255)**0.67;
im2 = np.uint8(im2);
imshow(im2, 'Correccion $\gamma=0.67$', 'gray');

im2 = 255*(np.float64(im)/255)**2.5;
im2 = np.uint8(im2);
imshow(im2, 'Correccion $\gamma=2.5$', 'gray');

## Estiramiento de contraste

El contraste de una imagen se puede modificar de manera simple al aplicar una fucncion punto a punto sobre los pixeles, mapeando asi sus valores a los deseados. 

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1iSIRuA6YmLylfFDp7yeQjYTvRdIWkUX6' -O washed_out_pollen_image.tif

im = plt.imread('washed_out_pollen_image.tif');
imshow(im, 'Original - bajo contraste', 'gray', [0, 255])


En este ejemplo definiremos una funcion de transformacion que duplica en contraste en la zona media del rango de valores, mientras que comprime a la mitad el contraste en las zonas bajas y altas. 

In [ ]:
def _curva(x):
    if x < 64:
        return (x * 0.5) * 255/319
    elif x < 192:
        return (2*x - 96) * 255/319
    else:
        return (0.5*x + 192) * 255/319
curva = np.vectorize(_curva)

x = np.arange(0, 255)
plt.figure()
plt.plot(x, curva(x))
plt.title('Curva de contraste')
plt.show()


In [ ]:
im2 = np.uint8(curva(np.float64(im)));
imshow(im2, 'Imagen con mayor contraste', 'gray', [0, 255]);


## Planos de bit

La informacion de una imagen se puede descomponer desde el rango de bits que componen sus pixeles. La intensidad de una imagen de 8 bits, cuyos pixeles toman valores entre 0 y 255, se puede comprender desde el punto de vista de la informacion binaria dispuesta en planos que la compone. Así los 8 bits se comprenden como 8 capas con diferentes valores de intensidad. 

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1jMrap9x1L8rCgl4xw4hGXzDznZ4-hT1V' -O dollar.tif

im = plt.imread('dollar.tif');
imshow(im, 'Original', 'gray')

In [ ]:
planos_de_bits = np.unpackbits(im).reshape(im.shape + (8,))
for i in range(8):
    imshow(planos_de_bits[:,:,i], 'bit {}'.format(8-i), 'gray')

In [ ]:
im2 = planos_de_bits[:,:,0]*2**7 + planos_de_bits[:,:,1]*2**6
imshow(im2, 'Bits 8 y 7', 'gray', [0, 255])

im2 = planos_de_bits[:,:,0]*2**7 + planos_de_bits[:,:,1]*2**6 + planos_de_bits[:,:,2]*2**5
imshow(im2, 'Bits 8, 7 y 6', 'gray', [0, 255])

im2 = planos_de_bits[:,:,0]*2**7 + planos_de_bits[:,:,1]*2**6 + planos_de_bits[:,:,2]*2**5 + planos_de_bits[:,:,3]*2**4
imshow(im2, 'Bits 8, 7, 6 y 5', 'gray', [0, 255])

## Procesamiento de histograma

El prosesamiento de histograma corresponde a transformaciones de intensidad que en vez de aplicarse a cada pixel de forma intependiente, se aplica a cada pixel dependiendo del resto mediante la distribucion acumulada de la informacion. 

In [ ]:
im = plt.imread('washed_out_pollen_image.tif')
imshow(im, 'Imagen Original', 'gray', [0, 255])
plt.figure()
plt.hist(im.ravel(), 256, [0,255])
plt.title('Histograma Original')
plt.show()

### Extension histograma

La extension del histograma mapea el valor minimo a 0 y el maximo a 255, mientras que los valores intermedios se ajustan a esta distribucion del valores. 


In [ ]:
im = np.float64(im)
im = im - np.min(im)
im = 255.0 * im / np.max(im)
im = np.uint8(im)
imshow(im, 'Imagen 2', 'gray', [0, 255])
plt.figure()
plt.hist(im.ravel(), 256, [0,255])
plt.title('Histograma extendido')
plt.show()

### Ecualizacion de histograma

La ecualizacion de histograma cambia la forma de este, de forma que podemos mapear cualquier distribucion a nuestra imagen. En este ejemplo veremos como aplicar una distribucion uniforme de valores.

In [ ]:
im_aux = im.ravel()
j = np.argsort(im_aux)
r = np.round(np.linspace(0.0, 255.0, j.shape[0]))
im_aux[j] = r
im_aux = im_aux.reshape(im.shape)
imshow(im_aux, 'Im2', 'gray', [0, 255])
plt.figure()
plt.hist(im.ravel(), 256, [0,255])
plt.title('Histograma ecualizado')
plt.show()

## Ejemplo 
 
 En este ejemplo veremos como la combinacion de las tecnicas nos permite mejorar imagenes. Este ejemplo puede demorar mas en ejecutarse ya que el tamaño de la imagen es considerablemente mayor. 

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1j0VcN-PTrk7TVoD1OewZqy_gASH2z3fL' -O IMG_20190724_170803.jpg

im = plt.imread('IMG_20190724_170803.jpg')
imshow(im, 'Original')

In [ ]:
R = im[:,:,0]
G = im[:,:,1]
B = im[:,:,2]

R_aux = R.ravel()
j = np.argsort(R_aux)
r = np.round(np.linspace(0.0, 255.0, j.shape[0]))
R_aux[j] = r
R_aux = R_aux.reshape(im.shape[0:2])

G_aux = G.ravel()
j = np.argsort(G_aux)
r = np.round(np.linspace(0.0, 255.0, j.shape[0]))
G_aux[j] = r
G_aux = G_aux.reshape(im.shape[0:2])

B_aux = B.ravel()
j = np.argsort(B_aux)
r = np.round(np.linspace(0.0, 255.0, j.shape[0]))
B_aux[j] = r
B_aux = B_aux.reshape(im.shape[0:2])

im2 = im.copy()
im2[:,:,0] = R_aux
im2[:,:,1] = G_aux
im2[:,:,2] = B_aux

imshow(im2, 'Imagen ecualizada')

In [ ]:
R = np.float64(im[:,:,0])
G = np.float64(im[:,:,1])
B = np.float64(im[:,:,2])

R = 255 * ((R/255)**0.6)
G = 255 * ((G/255)**0.5)
B = 255 * ((B/255)**0.55)

im2 = im.copy()
im2[:,:,0] = np.uint8(R)
im2[:,:,1] = np.uint8(G)
im2[:,:,2] = np.uint8(B)

imshow(im2, 'Imagen ajuste gamma')